In [2]:
from collections import defaultdict as ddict

In [3]:
datadict = ddict( list )

In [4]:
datadict[ 'carvone'].append( 1 )

In [5]:
from __future__ import division
import numpy as np
import h5py
import os
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [69]:
class Session:
    
    def __init__(self, data):
        self.data = data
        self.trialsdataset = self.data['Trials']
        self.trials = self.trialsdataset['trialNumber']
        self.numtrials = len(self.trials)
        self.results = self.trialsdataset['result']
        self.proportion_correct = 0
        self.hits = 0
        self.misses = 0
        self.rewards = []
        self.pc = self.calculate_percent_correct()
        
        self.flows0 = self.trialsdataset['olfas:olfa_0:mfc_1_flow']
        self.flows1 = self.trialsdataset['olfas:olfa_1:mfc_1_flow']
        
        self.odors0 = self.trialsdataset['olfas:olfa_0:odor']
        self.odors1 = self.trialsdataset['olfas:olfa_1:odor']
        
#         try:
#             self.odors0 = self.trialsdataset['olfas:olfa_0:odor']
#             self.odors1 = self.trialsdataset['olfas:olfa_1:odor']
#         except AttributeError:
#             pass
    
        self.context = self.trialsdataset['context']
        
        self.concmatdict = {1:{}, 2:{}}
        carvonedict = {None:{}, '(+)-carvone':{}, '(-)-carvone':{}}
        limonenedict = {None:[], '(+)-limonene':[], '(-)-limonene':[]}
        
        for key in self.concmatdict.keys():
            for carkey in carvonedict.keys():
                carvonedict[carkey] = limonenedict
            self.concmatdict[key] = carvonedict
                    
       
        print 'dictionary initialization done'
                
        
        self.make_concentration_bins()
        return

        
    def calculate_percent_correct(self):
        for i in xrange(len(self.results)):
            if self.results[i] == 1 or self.results[i]==2:
                self.rewards.append(1)
                self.hits +=1
            else:
                self.rewards.append(0)
            if self.results[i] > 4:
                self.misses +=1
            
        self.proportion_correct = self.hits/(self.numtrials-self.misses)
        self.pc = 100*self.proportion_correct
        return self.pc
    
#     def make_mixture_array(self):
#         for trial in range(self.numtrials):
            

    def make_concentration_bins(self):
        carvoneflows = [0, 10, 30, 100]
        limoneneflows = [0, 10, 30, 100]
        contextlist = [1, 2]
        carvonelist = ['(+)-carvone', None, '(-)-carvone']
        limonenelist = ['(+)-limonene', None, '(-)-limonene']

#         carvoneflows = [30]
#         limoneneflows = [30]
#         contextlist = [2]
#         carvonelist = ['(+)-carvone']
#         limonenelist = ['(+)-limonene']
        
        for trial in range(self.numtrials):
            if self.odors0[trial] == None:
                self.flows0[trial] = 0
            if self.odors1[trial] == None:
                self.flows1[trial] = 0
                
        for cont, key in zip(contextlist, self.concmatdict.keys()):
            print cont, key
            
            for carv, carvkey in zip(carvonelist, self.concmatdict[key].keys()):
                print carv, carvkey
                for lim, limkey in zip(limonenelist, self.concmatdict[key][carvkey].keys()):
                    print lim, limkey
                    counter = 0
                    concmat = np.zeros((4, 4, 3)) # first dimension is hits, second num trials, third percent correct
                    
                    for i in range(len(carvoneflows)):
                        carcon = carvoneflows[i]
                        for j in range(len(limoneneflows)):
                            limcon = limoneneflows[j]
                            
                            for trial in range(self.numtrials):
                                if self.context[trial] == cont and self.odors0[trial] == carv and self.odors1[trial] == lim \
                                and self.flows0[trial] == carcon and self.flows1[trial] == limcon:
                                    counter +=1
                                    concmat[i][j][1] += 1
                                    if self.rewards[trial] == 1:
                                        concmat[i][j][0] += 1
                                        
                    for a in range(4):
                        for b in range(4):
                            if concmat[a][b][1]:
                                print concmat[a][b][0], concmat[a][b][1]
                                concmat[a][b][2] = 100.*concmat[a][b][0]/concmat[a][b][1]
                                print concmat[a][b][2], a, b, key, carvkey, limkey

                            else:
                                concmat[a][b][2] = 0

                            self.concmatdict[key][carvkey][limkey] = concmat # only taking the last calculated value of concmat[a][b][2]
                            # also only calculating values for the 4 comb of carvone+lim, not when either one equals None
                    del concmat
        return self.concmatdict
                           
                            
    
data = h5py.File('C:\VoyeurData\mouse8283_sess1_D2016_12_9T13_11_51.h5')
print day1.concmatdict        
day1 = Session(data)
# print day1.proportion_correct 
# print '{0:.2f}'.format(day1.pc)
# print day1.concmatdict
#plt.imshow(day1.concmat, interpolation='nearest', cmap=plt.cm.ocean)
# plt.figure()
# plt.plot(flows, day1.limonene_means)
    

# print session.numtrials
# print 'Percent correct:', session.pc
# print '{0:.2f}'.format(session.pc),'%;', session.hits,'H;', session.misses,'M;', session.numtrials,'T'

# def overall_performance(instance.odorset): # hits, misses, percent correct, etc from all trials of same odorset
#     # should this go into a class?
#     for i in xrange(len(instance.odorsets)):
#         if instance.context[i] == 1:
#         if instance.context[i] == 2:

{1: {'(+)-carvone': {'(+)-limonene': array([[[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [ 26.        ,  36.        ,  72.22222222],
        [  0.        ,   0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]]]), None: array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0., 